<a href="https://colab.research.google.com/github/shitkov/movements/blob/main/neck_static.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision import models

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import cv2
from google.colab.patches import cv2_imshow
import mediapipe as mp

from os import listdir
from os.path import isfile, join

In [4]:
path_0 = [
    '/content/drive/MyDrive/ml/neck/static_correct_1.MOV'
]

In [5]:
path_0_90 = ['/content/drive/MyDrive/ml/neck/static_me.mp4']

In [6]:
path_1 = [
    '/content/drive/MyDrive/ml/neck/static_correct_2.mp4',
    '/content/drive/MyDrive/ml/neck/static_correct_3.MOV',
    '/content/drive/MyDrive/ml/neck/static_head_goes_forward_1.mp4',
    '/content/drive/MyDrive/ml/neck/static_head_goes_forward_2.mp4',
    '/content/drive/MyDrive/ml/neck/static_nearest_shoulder_rises_1.mp4',
    '/content/drive/MyDrive/ml/neck/static_nearest_shoulder_rises_2.mp4',
    '/content/drive/MyDrive/ml/neck/static_shoulder_rises_1.mp4',
    '/content/drive/MyDrive/ml/neck/static_shoulder_rises_2.mp4'

]

In [7]:
key_points = [0, 7, 8, 11, 12, 23, 24, 25, 26]

In [8]:
def get_points(path, key_points):
    vidcap = cv2.VideoCapture(path)
    points_list = []
    for _ in tqdm(range(int(cv2.VideoCapture(path).get(cv2.CAP_PROP_FRAME_COUNT)))):
        _, image = vidcap.read()
        try:
            with mp.solutions.pose.Pose(static_image_mode=False, min_detection_confidence=0.3, model_complexity=1) as pose:
                results = pose.process(image)
            points = []
            for i, point in enumerate(results.pose_world_landmarks.landmark):
                if i in key_points:
                    points.append([point.x, point.y, point.z])
            points_list.append(np.array(points).reshape(-1))
        except:
            # points_list.append(np.zeros(len(key_points) * 3))
            pass
    return np.array(points_list)

In [9]:
def get_points_rotate(path, key_points):
    vidcap = cv2.VideoCapture(path)
    points_list = []
    for _ in tqdm(range(int(cv2.VideoCapture(path).get(cv2.CAP_PROP_FRAME_COUNT)))):
        _, image = vidcap.read()
        image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        try:
            with mp.solutions.pose.Pose(static_image_mode=False, min_detection_confidence=0.3, model_complexity=1) as pose:
                results = pose.process(image)
            points = []
            for i, point in enumerate(results.pose_world_landmarks.landmark):
                if i in key_points:
                    points.append([point.x, point.y, point.z])
            points_list.append(np.array(points).reshape(-1))
        except:
            # points_list.append(np.zeros(len(key_points) * 3))
            pass
    return np.array(points_list)

In [10]:
points_0 = get_points(path_0[0], key_points)

100%|██████████| 505/505 [01:46<00:00,  4.73it/s]


In [11]:
points_0_90 = get_points_rotate(path_0_90[0], key_points)

100%|██████████| 787/787 [02:42<00:00,  4.84it/s]


In [12]:
points_0 = points_0.tolist()

In [13]:
points_0_90 = points_0_90.tolist()

In [14]:
points_0 += points_0_90

In [15]:
points_1_list = []

for path in path_1:
    points_1_list.append(get_points(path, key_points))

100%|██████████| 338/338 [01:06<00:00,  5.10it/s]


In [16]:
points_1 = []
for arr in points_1_list:
    points_1 += arr.tolist()

In [17]:
import pickle

with open('/content/drive/MyDrive/ml/neck/' + 'points_0.p', 'wb') as f:
    pickle.dump(points_0, f)

with open('/content/drive/MyDrive/ml/neck/' + 'points_1.p', 'wb') as f:
    pickle.dump(points_1, f)

In [18]:
data = pd.DataFrame()
data['points'] = points_0 + points_1

data['label'] = [0] * len(points_0) + [1] * len(points_1)

In [19]:
data = data.sample(frac=1)

In [20]:
data = data.reset_index(drop = True)

In [21]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, stratify=list(data['label']), test_size = 0.2)

In [22]:
train.to_csv('/content/drive/MyDrive/ml/neck/static_train.csv', index=False)
test.to_csv('/content/drive/MyDrive/ml/neck/static_test.csv', index=False)

In [23]:
class CustomDataset(Dataset):
    
    def __init__(self, points, labels):
        self.points = points
        self.labels = labels
        
    def __getitem__(self, index):
        return self.points[index], self.labels[index]
        
    def __len__ (self):
        return len(self.points)

In [24]:
train_dataset = CustomDataset(torch.FloatTensor(list(train['points'])), torch.FloatTensor(list(train['label'])))

In [87]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

In [88]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [89]:
class SimleNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimleNetwork,self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim * 2)
        self.linear2 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.linear3 = nn.Linear(hidden_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, output_dim)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.linear3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.out(x)
        return x

In [90]:
input_dim = len(list(train['points'])[0])
hidden_dim = 128
output_dim = 1

In [91]:
loss_fn = nn.BCEWithLogitsLoss()

In [92]:
model = SimleNetwork(input_dim, hidden_dim, output_dim)

In [93]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)

SimleNetwork(
  (linear1): Linear(in_features=27, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
)

In [94]:
model.to(device)

SimleNetwork(
  (linear1): Linear(in_features=27, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
)

In [95]:
optimizer = optim.AdamW(model.parameters(), lr=0.0001)

In [96]:
def eval(model, x, y):
    model.eval()
    with torch.no_grad():
        X = torch.FloatTensor(x)
        y = torch.FloatTensor(y)
        X = X.to(device)
        y_test_pred = model(X)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_test_pred = torch.round(y_test_pred)
        y_test_pred = y_test_pred.reshape(1, -1).cpu().detach().numpy()[0]

    return y_test_pred

In [97]:
test_points = torch.FloatTensor(np.array(list(test['points'])))
test_labels = torch.FloatTensor(np.array(list(test['label'])))

In [98]:
for epoch in range(100):
    model.train()
    for data in tqdm(train_loader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        try:
            outputs = torch.sigmoid(model(inputs))
            loss = loss_fn(outputs, labels.reshape(-1, 1))
            loss.backward()
            optimizer.step()
        except:
            pass

    predictions = eval(model, test_points, test_labels)
    print(epoch, f1_score(test_labels.tolist(), predictions.tolist(), average='macro'))

100%|██████████| 102/102 [00:00<00:00, 455.86it/s]


0 0.8533034538958191


100%|██████████| 102/102 [00:00<00:00, 446.21it/s]


1 0.8595754661208319


100%|██████████| 102/102 [00:00<00:00, 472.53it/s]


...

97 0.9971685971685972


100%|██████████| 102/102 [00:00<00:00, 354.86it/s]


98 0.9929431795917447


100%|██████████| 102/102 [00:00<00:00, 345.11it/s]


99 0.9985828370071049


In [100]:
# Save NN
path = '/content/drive/MyDrive/ml/neck/neck_v0.pt'
torch.save(model.state_dict(), path)

In [101]:
model_neck = SimleNetwork(27, 128, 1)

In [102]:
model_neck.load_state_dict(torch.load(path))

<All keys matched successfully>

In [104]:
predicts = eval(model_neck, test_points, test_labels)

In [105]:
f1_score(test_labels.tolist(), predicts.tolist(), average='macro')

0.9985828370071049